In [8]:
# 다대다 문자 단위 RNN(Char RNN) - 워드 임베딩

# 모듈 불러오기
import torch
import torch.nn as nn
import torch.optim as optim

### 데이터 전처리 ###
# 문자열 데이터
sentence = "Repeat is the best medicine for memory".split()

# 단어집합
vocab = list(set(sentence))
print(vocab)

# 정수 인코딩
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}
word2index['<unk>'] = 0
print(word2index)
print(word2index['memory'])

# 예측결과 보기 위한 문자 인코딩
index2word = {v: k for k, v in word2index.items()}
print(index2word)
print(index2word[7])

# 정수 인코딩 + 데이터 배치 차원 추가 + 데이터셋 생성
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 정수 인코딩
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq
X, Y = build_data(sentence, word2index)
print(X)
print(Y)

# 하이퍼파라미터 초기화
input_size = 5 # 입력의 차원
hidden_size = 20 # 은닉상태 크기
vocab_size = len(word2index) # 단어장의 크기는 임베딩 층, 최종 출력층에 사용. <unk> 토큰도 크기에 포함

['for', 'the', 'best', 'medicine', 'is', 'Repeat', 'memory']
{'for': 1, 'the': 2, 'best': 3, 'medicine': 4, 'is': 5, 'Repeat': 6, 'memory': 7, '<unk>': 0}
7
{1: 'for', 2: 'the', 3: 'best', 4: 'medicine', 5: 'is', 6: 'Repeat', 7: 'memory', 0: '<unk>'}
memory
tensor([[6, 5, 2, 3, 4, 1]])
tensor([[5, 2, 3, 4, 1, 7]])


In [11]:
### RNN ###
# RNN셀 구현 - 워드 임베딩
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=input_size) # 워드 임베딩
        self.rnn_layer = nn.RNN(input_size, hidden_size, batch_first=batch_first) # RNN셀 구현 - 은닉층 1개 사용
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력층 구현. 원-핫 벡터의 크기를 가져야함 or 단어 집합의 크기만큼 가져야함
    def forward(self, x):
        output = self.embedding_layer(x)
            # 1. 임베딩 층
            # 크기변화 : (배치 크기, 시퀀스 길이) -> (배치 크기, 시퀀스 길이, 임베딩 차원)
        output, hidden = self.rnn_layer(output)
            # 2. RNN 층
            # 크기변화 : (배치 크기, 시퀀스 길이, 임베딩 차원)
                # -> output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden(1, 배치 크기, 은닉층 크기)
        output = self.linear(output)
            # 3. 최종 출력층
            # 크기변화 : (배치 크기, 시퀀스 길이, 은닉층 크기) -> (배치 크기, 시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))
            # 4. view를 통해 배치 차원 제거
            # 크기변화 : (배치 크기, 시퀀스 길이, 단어장 크기) -> (배치 크기 * 시퀀스 길이, 단어장 크기)
    
model = Net(vocab_size, input_size, hidden_size, batch_first = True)
output = model(X)
print(output)
print(output.shape) # 3차원 텐서
    # RNN셀의 리턴 : (시퀀스 길이, 입력층의 크기)

# 최적화 함수, 손실함수 정의
optimizer = optim.Adam(params=model.parameters())
loss_function = nn.CrossEntropyLoss()

# 문자 인코딩
decode = lambda y: [index2word.get(x) for x in y]

# 학습
for step in range(201):
    # 최적화 함수 초기화
    optimizer.zero_grad() # 경사 초기화
    
    # 학습
    output = model(X) # 순방향 전파
    loss = loss_function(output, Y.view(-1)) # 손실값 계산
    loss.backward() # 역방향 전파
    optimizer.step() # 매개변수 업데이트
    
    # 모델 예측값 확인
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f}".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

tensor([[ 0.0990, -0.4268,  0.0978, -0.0624, -0.1736, -0.1880, -0.0745, -0.1346],
        [-0.1396, -0.4790,  0.5572,  0.0996, -0.3084, -0.0512, -0.0466,  0.1305],
        [-0.0487, -0.1910,  0.4054,  0.0541, -0.3248, -0.1528,  0.0312, -0.1771],
        [-0.0931, -0.2733,  0.4366, -0.0723, -0.3181, -0.0594, -0.0105, -0.1895],
        [-0.1997, -0.2435,  0.4135,  0.1736, -0.3023, -0.2035,  0.0191,  0.1516],
        [-0.0698, -0.0254,  0.0008, -0.1907, -0.1969, -0.2603,  0.0052, -0.3606]],
       grad_fn=<ViewBackward0>)
torch.Size([6, 8])
[01/201] 2.1161
Repeat <unk> the the the the Repeat

[41/201] 1.5231
Repeat the the best medicine for memory

[81/201] 0.8634
Repeat is the best medicine for memory

[121/201] 0.4246
Repeat is the best medicine for memory

[161/201] 0.2335
Repeat is the best medicine for memory

[201/201] 0.1398
Repeat is the best medicine for memory

